# Team (◕‿◕✿)'s Project Process

## 1. Scraping and cleaning our data
First, we narrow down our comments to AskReddit comments made in the time range 10/01/2014 - 10/01/2015. 

The URL for the above filtering is: https://www.reddit.com/r/AskReddit/search?sort=comments&q=timestamp%3A1410739200..1411171200&restrict_sr=on&syntax=cloudsearch

Breaking down the above URL:
* **/r/AskReddit**: filter to subreddit AskReddit
* **sort=comments**: the search results will be sorted based on number of comments
* **timestamp%3A1410739200..1411171200&**: restrict to 10/01/2014 00:00:00 to 10/01/2015 00:00:00, which are translated into epochs in the URL.
* **syntax=cloudsearch**: reddit's search is infamously bad. One reason is that its regular search syntax, Lucene, doesn't allow for timestamp search. We thus choose CloudSearch as our syntax instead, which allows us to use features (like timestamp searches) that Lucene does not allow.

reddit will return at most 10

In [12]:
import praw
r = praw.Reddit('Getting comments for CS109 Final Project, team (◕‿◕✿)')
submission = r.get_submission(submission_id='2gg9vw')

In [13]:
%%time
submission.replace_more_comments(limit=None, threshold=0)

CPU times: user 1.98 s, sys: 133 ms, total: 2.11 s
Wall time: 8min 24s


In [28]:
from pprint import pprint
all_comments = submission.comments
pprint (vars(all_comments[0]))

{'_has_fetched': True,
 '_has_fetched_replies': True,
 '_info_url': u'https://api.reddit.com/api/info/',
 '_replies': [<praw.objects.Comment object at 0x1039bcf50>,
 '_submission': <praw.objects.Submission object at 0x1039bc950>,
 '_underscore_names': [u'replies'],
 '_uniq': None,
 'approved_by': None,
 'archived': True,
 'author': Redditor(user_name='Kensarowiwa'),
 'author_flair_css_class': None,
 'author_flair_text': None,
 'banned_by': None,
 'body': u'Omar Little ',
 'body_html': u'<div class="md"><p>Omar Little </p>\n</div>',
 'controversiality': 0,
 'created': 1410820415.0,
 'created_utc': 1410791615.0,
 'distinguished': None,
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'id': u'ckiw43d',
 'json_dict': None,
 'likes': None,
 'link_id': u't3_2gg9vw',
 'mod_reports': [],
 'name': u't1_ckiw43d',
 'num_reports': None,
 'parent_id': u't3_2gg9vw',
 'reddit_session': <praw.Reddit object at 0x1047e4390>,
 'removal_reason': None,
 'report_reasons': None,
 'saved': False,
 'score': 137,
